In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import sqlalchemy

from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from joblib import load
import joblib
from lightgbm.sklearn import LGBMClassifier,LGBMRegressor
from hdfs import InsecureClient
from sklearn.model_selection import  KFold

In [3]:
import pyhive

In [4]:

# HDFS 설정
hdfs_url = 'http://43.201.229.213:50070'
hdfs_path = '/test/seoul_pollution_2023-10-23-00_29.json'

# HDFS 클라이언트 생성
client = InsecureClient(hdfs_url, user='ubuntu')

# HDFS에서 데이터 읽기
with client.read(hdfs_path) as reader:
    data = reader.read()

data_json = json.loads(data.decode('unicode_escape'))
df = pd.DataFrame(data_json)



ConnectTimeout: HTTPConnectionPool(host='43.201.229.213', port=50070): Max retries exceeded with url: /webhdfs/v1/test/seoul_pollution_2023-10-23-00_29.json?user.name=ubuntu&offset=0&op=OPEN (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7fc335864d60>, 'Connection to 43.201.229.213 timed out. (connect timeout=None)'))

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [ ]:
col= { 'spot': 'gu','hour':'time'}
df.rename(columns=col,inplace=True)
df = df[df['gu'] !='강북구']

In [ ]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day'] =df['date'].dt.day
df = df.drop(columns='date')

In [ ]:


# 파일에서 객체 로드
label_encoder = joblib.load('label_encoder_classes.joblib')

# 로드된 객체를 사용하여 변환
df['gu'] = label_encoder.transform(df['gu'])

# 또는 반대로 변환도 가능
# decoded_labels = label_encoder.inverse_transform(encoded_labels)



In [ ]:
pr

In [ ]:
df['time'] =df['time'].astype(int)

In [ ]:
df

In [ ]:
    model = load('./lgbm_model.pkl')
    preds = model.predict(df)

In [ ]:
print(preds)

In [ ]:
df['traffic'] = preds

In [ ]:
df 

In [ ]:
df['gu'] = label_encoder.inverse_transform(df['gu'])

In [ ]:
df

In [ ]:
df['gu'].unique()

In [ ]:


hdfs_paths = '/test/seoul_traffic_2023-10-23-00_29.json'
# HDFS 클라이언트 생성
client = InsecureClient(hdfs_url, user='ubuntu')

# HDFS에서 데이터 읽기
with client.read(hdfs_paths) as reader:
    data = reader.read()

data_json = json.loads(data.decode('unicode_escape'))
tr_df = pd.DataFrame(data_json)

In [ ]:
tr_df

In [ ]:
gu_df = pd.read_csv('./spot_data_guz.csv')

In [ ]:
cols ={ 'spot_num' : 'spot' }
gu_df.rename(columns=cols,inplace=True)

In [ ]:
tr_df = pd.merge(gu_df, tr_df, on=['spot'])

In [ ]:
tr_df.drop(columns='spot',inplace=True)

In [ ]:
tr_df.dtypes

In [134]:
tr_df = tr_df.groupby(['gu','date','hour'])[['vol']].sum().reset_index()

In [135]:
tr_df['gu'].unique()

array(['강남구', '강동구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구',
       '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구',
       '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'], dtype=object)

In [136]:
tr_df

,gu,date,hour,vol
0,강남구,20231023,08,30566
1,강동구,20231023,08,12920
2,강서구,20231023,08,18543
3,관악구,20231023,08,3531
4,광진구,20231023,08,14972
5,구로구,20231023,08,11437
6,금천구,20231023,08,14565
7,노원구,20231023,08,18830
8,도봉구,20231023,08,4253
9,동대문구,20231023,08,5005


In [137]:
new_df=pd.merge(tr_df,df, on=['gu'])

In [139]:
drop_cols = { 'time','year','month','day'}
new_df.drop(columns=drop_cols,inplace=True)

In [142]:
new_df['date']= pd.to_datetime(new_df['date'],format='%Y%m%d')

In [146]:
new_df['hour'] = new_df['hour'].astype(int)

In [147]:
new_df

,gu,date,hour,vol,PM10,PM25,O3,NO2,CO,SO2,traffic
0,강남구,2023-10-23,8,30566,25.0,14.0,0.003,0.029,0.4,0.003,16647.517134
1,강동구,2023-10-23,8,12920,40.0,23.0,0.002,0.034,0.9,0.003,14759.354040
2,강서구,2023-10-23,8,18543,27.0,11.0,0.003,0.034,0.7,0.003,14848.352543
3,관악구,2023-10-23,8,3531,26.0,11.0,0.008,0.033,0.6,0.003,14366.561273
4,광진구,2023-10-23,8,14972,25.0,13.0,0.006,0.027,0.6,0.002,14332.193921
5,구로구,2023-10-23,8,11437,23.0,15.0,0.003,0.023,0.4,0.002,14710.803085
6,금천구,2023-10-23,8,14565,23.0,11.0,0.003,0.032,0.7,0.002,14722.341818
7,노원구,2023-10-23,8,18830,23.0,13.0,0.002,0.030,0.7,0.003,14254.841672
8,도봉구,2023-10-23,8,4253,28.0,20.0,0.006,0.026,0.7,0.003,14242.451624
9,동대문구,2023-10-23,8,5005,29.0,14.0,0.003,0.032,0.6,0.003,14070.398665
